In [ ]:
import numpy as np
from adaptive_latents.input_sources import LDS
import adaptive_latents
from adaptive_latents import VJF
from tqdm.autonotebook import trange

import matplotlib.pyplot as plt


In [ ]:
rng = np.random.default_rng(1)

In [ ]:
transitions_per_rotation = 30
radius = 5
_, Y, stim = LDS.nest_dynamical_system(1000, u_function=lambda **_: np.zeros(3), rng=rng)

one_step_distance = np.linalg.norm(Y[0] - Y[1])

predictor = VJF(latent_d=2, rng=rng)

# mu = predictor.offline_run_on(Y, show_tqdm=True)



In [ ]:
print(Y.sum())



In [ ]:
predictions = []
for i in trange(transitions_per_rotation+1):
    prediction = predictor.partial_fit_transform(adaptive_latents.ArrayWithTime([[i]], Y.t[-1]), stream='pred_obs_space')
    predictions.append(prediction)

trajectory = np.squeeze(predictions)


In [ ]:
plt.plot(Y[:, 0], Y[:, 1])
# plt.plot([initial_point[0, 0], X_hat[0, 0]], [initial_point[0, 1], X_hat[0, 1]], '--.', color='C2')
plt.plot(trajectory[:, 0], trajectory[:, 1], '.-')
plt.axis('equal')
plt.show()


In [ ]:
predictor.show_nstep_cloud(0)